In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.gridspec import GridSpec
from matplotlib import patches

In [ ]:
def lpf(x, min, max, fc, fs):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
    
    T =1/fs
    wd = 2*np.pi*fc
    wp = (2/T)*np.tan(wd*T/2)
    n = np.arange(0, 361)
    ohm = np.pi*n*fc/(fs*360)
    
    a0 = (wp**2 * T**2)/(4+2*np.sqrt(2)*wp*T+ wp** 2* T**2)
    a1 = (2 * wp**2 * T**2)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)
    a2 = (wp**2 * T**2)/(4+2*np.sqrt(2)*wp*T+ wp** 2* T**2)
    b0 = 1
    b1 = ((2 * wp**2 * T**2)-8)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)
    b2 = (4-2*np.sqrt(2)*wp*T+ wp**2 * T**2)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)

    # Filter
    for n in range(max-min):
        y[n] = -b1*y[n-1]-b2*y[n-2]+a0*x[n]+a1*x[n-1]+a2*x[n-2]
        
   
    #bilinier transform
    top_r = a0+a1*np.cos(ohm)+a2*np.cos(2*ohm)
    top_i = -a1*np.sin(ohm)-a2*np.sin(2*ohm) 
    bot_r = b0+b1*np.cos(ohm)+b2*np.cos(2*ohm)
    bot_i = -b1*np.sin(ohm)-b2*np.sin(2*ohm)

    H = np.sqrt((top_r**2+top_i**2)/(bot_r**2+bot_i**2))

    return y, H

def hpf(x, min, max, fc, fs):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
   

    T =1/fs
    wd = 2*np.pi*fc
    wp = (2/T)*np.tan(wd*T/2)
    n = np.arange(0, 361)
    ohm = np.pi*n*fc/(fs*360)
    
    a0 = (4)/(4+2*np.sqrt(2)*wp*T+ wp** 2* T**2)
    a1 = (-8)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)
    a2 = (4)/(4+2*np.sqrt(2)*wp*T+ wp** 2* T**2)
    b0 = 1
    b1 = ((2 * wp**2 * T**2)-8)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)
    b2 = (4-2*np.sqrt(2)*wp*T+ wp**2 * T**2)/(4+2*np.sqrt(2)*wp*T+ wp**2 * T**2)

    # Filter
    for n in range(delta):
        y[n] = -b1*y[n-1]-b2*y[n-2]+a0*x[n]+a1*x[n-1]+a2*x[n-2]
        

    #bilinier transform
    top_r = a0+a1*np.cos(ohm)+a2*np.cos(2*ohm)
    top_i = -a1*np.sin(ohm)-a2*np.sin(2*ohm) 
    bot_r = b0+b1*np.cos(ohm)+b2*np.cos(2*ohm)
    bot_i = -b1*np.sin(ohm)-b2*np.sin(2*ohm)

    H = np.sqrt((top_r**2+top_i**2)/(bot_r**2+bot_i**2))

    return y, H

def div(x, min, max):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
    for n in range(delta):
        y[n] = 0.125*(2*x[n]+x[n-1]-x[n-3]-2*x[n-4])

    return y
    
def squar(x):
    return x**2

def miw(x, min, max):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
    N = 40
    for n in range(delta):
        for i in range(1, N):
            y[n] += x[n-N+i]
    
    return y

def trunc(x,min, max):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
    peak = np.max(x)
    for i in range(delta):
        if x[i] > 0.35*peak:
            y[i] = peak
        else:
            y[i] = 0

    return y

def base(x, data, min, max):
    delta = max-min
    y = np.zeros(2*(max-min)+1)
    p = np.max(data)
    q = np.min(data)
    for n in range(delta):
        if x[n+30] <= 0:
            y[n] = q
        else :
            y[n] = p

    return y 

In [ ]:
def qrs(min='0', max='5', fs='360', data='ecg100.txt', fl=16, fh=52):
    cont_x1 = np.loadtxt(data, usecols= 1, dtype=float)
    cont_x2 = np.loadtxt(data, usecols=2, dtype=float)

    fs  = int(fs)
    min = int(min)
    max = int(max)

    min = min*fs
    max = (max*fs)+1

    the_l = 2*np.pi*fl/fs
    the_h = 2*np.pi*fh/fs
    
    x = np.zeros(2*(max-min)+1)

    x[:max-min] = cont_x1[min:max]

    y1, H_lpf = lpf(x, min, max, fl, fs)

    y2, H_hpf = hpf(y1, min, max, fh, fs)

    y3 = div(y2, min, max)

    y4 = squar(y3)

    y5 = miw(y4, min, max)

    y6 = trunc(y5, min, max)

    y7 = base(y6, x, min, max)

    H = H_lpf + H_hpf

    t = np.arange(min,max)

    fig, ax = plt.subplots(4,1, figsize=(20, 9))

    # Data Output
    ax[0].plot(t/fs, x[t-min], color='r', label='ECG')
    ax[0].plot(t/fs, y7[t-min], color='g', label='QRS Detection')
    ax[0].set_title('ECG Signal', color='b', fontsize='xx-large')
    ax[0].set_ylabel('Amplitude (V)', fontsize='large')
    ax[0].set_xlabel('Time (s)', fontsize='large')
    ax[0].set_xlim(min/fs,max/fs)
    ax[0].set_ylim(np.min(x),np.max(x))
    ax[0].legend(loc='upper right')
    ax[0].grid()

    ax[1].plot(t/fs, y2[t-min], color='r', label='BPF')
    ax[1].plot(t/fs, y3[t-min], color='k', label='Dev')
    ax[1].set_title('Bandpass Filter and Derivative', color='b', fontsize='xx-large')
    ax[1].set_ylabel('Amplitude (V)', fontsize='large')
    ax[1].set_xlabel('Time (s)', fontsize='large')
    ax[1].set_xlim(min/fs,max/fs)
    ax[1].set_ylim(np.min(y2),np.max(y2))
    ax[1].legend(loc='upper right')
    ax[1].grid()

    ax[2].plot(t/fs, y4[t-min], color='r', label='Squaring')
    ax[2].set_title('Squaring', color='b', fontsize='xx-large')
    ax[2].set_ylabel('Amplitude (V)', fontsize='large')
    ax[2].set_xlabel('Time (s)', fontsize='large')
    ax[2].set_xlim(min/fs,max/fs)
    ax[2].set_ylim(np.min(y4),np.max(y4))
    ax[2].legend(loc='upper right')
    ax[2].grid()

    ax[3].plot(t/fs, y5[t-min], color='r', label='MW output')
    ax[3].plot(t/fs, y6[t-min], color='g', label='QRS Window')
    ax[3].set_title('Moving Window Integrator and QRS Window', color='b', fontsize='xx-large')
    ax[3].set_ylabel('Amplitude (V)', fontsize='large')
    ax[3].set_xlabel('Time (s)', fontsize='large')
    ax[3].set_xlim(min/fs,max/fs)
    ax[3].legend(loc='upper right')
    ax[3].grid()

    fig.tight_layout()


In [ ]:
widgets.interact(qrs, min='0', max='5', fs='360', data=[('data 1', 'ecg100.txt'),('data 2', 'ecg102.txt')
                , ('data 3', 'ecg104.txt'), ('data 4', 'ecg106.txt'), ('data 5', 'ecg108.txt')], fl=(0, 180), fh=(0, 180))

interactive(children=(Text(value='0', description='min'), Text(value='5', description='max'), Text(value='360'…

<function __main__.qrs(min='0', max='5', fs='360', data='ecg100.txt', fl=16, fh=52)>